## Functional Programming SIG 10.27.22

In [66]:
:opt no-lint
import Data.List
import GHC.Show
import Data.Char
import Control.Monad

### Algebraic Data Types Review

In [67]:
giveMeAValue :: Int -> Either Double Bool
giveMeAValue 0 = Right False
giveMeAValue _ = Left 10.01

In [68]:
parse :: String -> Either String [String]
parse s 
    | length s > 20 = Left "Error! cannot parse strings longer than 10 characters"
    | null s = Left "Error! empty string doesn't count okay"
    | otherwise = Right (words s)

In [69]:
spicyFunc :: (a -> b, a) -> b
spicyFunc (f, x) = f x 

In [70]:
(True, True) :: (Bool, Bool)
(True, False) :: (Bool, Bool)
(False, True) :: (Bool, Bool)
(False, False) :: (Bool, Bool)

(True,True)

(True,False)

(False,True)

(False,False)

### The data and type keywords

In [71]:
data MySumType = First | Second -- ~= Either () () ~= Bool

data MyProductType = Product Bool String -- ~= (Bool, String)

data JustADouble = ThisIsMyDouble Double -- ~= (Double)

foo :: JustADouble
foo = ThisIsMyDouble 89.9

In [72]:
data PizzaTopping = Pepperoni | Cheese | Olives | Pudding 
    deriving (Show)

favoriteTopping :: PizzaTopping
favoriteTopping = Olives

print favoriteTopping

Olives

In [73]:
data Shape = Circle (Double, Double) Double | Triangle (Double, Double) (Double, Double) (Double, Double)

In [74]:
type Point = (Double, Double)
type Radius = Double
data Shape = Circle Point Radius | Triangle Point Point Point
    deriving Show

In [75]:
gimmeAShape :: Bool -> Shape
gimmeAShape True = Circle (10.3,10.5) 100.0
gimmeAShape False = Triangle (0.0, 1.1) (1.1, 2.2) (2.2, 3.3)

gimmeAShape True

Circle (10.3,10.5) 100.0

In [76]:
eatShape :: Shape -> Double
eatShape (Circle center radius) = radius
eatShape (Triangle _ (_,y) _) = y

eatShape (gimmeAShape False)

2.2

In [77]:
-- data Recipe =

#### Maybe

In [78]:
data Maybe a = Just a | Nothing 
    deriving (Show)

divide :: Int -> Int -> Int
divide x y = x `div` y

divide 10 5
divide 10 0

2

: 

In [79]:
divide' :: Int -> Int -> Maybe Int
divide' x 0 = Nothing
divide' x y = Just (x `div` y)

divide' 10 5
divide' 10 0

Just 2

Nothing

In [80]:
lameDivide :: Int -> Int -> Either String Int
lameDivide _ 0 = Left "no"
lameDivide 7 _ = Left "sorry I don't like 7"
lameDivide x y = Right (x `div` y)

lameDivide 9071 0
lameDivide 80 2

Left "no"

Right 40

In [81]:
:type Just
:type Nothing
:type Left
:type True
:type Pepperoni

Just :: forall a. a -> Maybe a

Nothing :: forall a. Maybe a

Left :: forall a b. a -> Either a b

True :: Bool

Pepperoni :: PizzaTopping

In [82]:
data Person = Person {name :: String, phone :: String, likesHaskell :: Bool}
    deriving Show
            
isaac :: Person
isaac = Person {name = "Isaac", phone = "12", likesHaskell = True}

phone isaac

"12"

In [83]:
:type name
:type phone
:type likesHaskell

name :: Person -> String

phone :: Person -> String

likesHaskell :: Person -> Bool

In [84]:
-- data Tree a 

In [85]:
-- insert :: Ord a => Tree a -> a -> Tree a

In [86]:
-- fromList :: Ord a => [a] -> Tree a

In [87]:
-- toList :: Tree a -> [a]

In [88]:
-- getValue :: Tree a -> Maybe a 

### Function Composition and Point Free Style

In [89]:
(.) :: (b -> c) -> (a -> b) -> (a -> c)
f . g = \x -> f (g x)

In [90]:
toList (fromList [8,2,4,1,6,5,7,3])
(toList . fromList) [8,2,4,1,6,5,7,3]

: 

In [91]:
map (`mod` 3 == 0) [1..100]

: 

In [92]:
map ((==0) . (`mod` 3)) [1..100]

[False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False,False,True,False]

In [93]:
count :: String -> Int
count xs = length (nub (map toUpper (filter isAlpha xs)))

count' :: String -> Int
count' xs = (length . nub . (map toUpper) . (filter isAlpha)) xs

In [94]:
map ((map toLower) . fst) [("Aldi", 100), ("Hyvee", 45), ("Walmart", 40)]

["aldi","hyvee","walmart"]

In [95]:
getFirst :: [a] -> a
getFirst xs = head xs

getFirst' :: [a] -> a
getFirst' = head

In [96]:
count'' :: String -> Int
count'' = length . nub . (map toUpper) . (filter isAlpha)

In [97]:
(sum . map snd) [("Once", 4), ("upon", 4), ("a", 1), ("time", 4)]

13

[pointfree style converter](http://pointfree.io/)

In [98]:
apply :: (a -> b) -> (c -> d) -> (a, c) -> (b, d)
apply f g (x,y) = (f x, g y)

apply reverse (map toUpper) ("hello", "world")

("olleh","WORLD")

In [99]:
 apply' = flip flip snd . (ap .) . flip flip fst . ((.) .) . flip . (((.) . (,)) .)
 
 apply' reverse (map toUpper) ("hello", "world")

("olleh","WORLD")

[source](https://wiki.haskell.org/Pointfree)

### Infix operators

In [100]:
(+-+) :: (Num a, Ord a) => a -> a -> a
x +-+ y
    | x > 0 = x - y
    | otherwise = x + y

In [104]:
-- ($) :: (a -> b) -> a -> b

In [102]:
reverse (nub (map toUpper "fooBarrBaz!"))
reverse $ nub (map toUpper "fooBarrBaz!")
reverse $ nub $ map toUpper "fooBarrBaz!"


"!ZRABOF"

"!ZRABOF"

"!ZRABOF"